# Bookcase Data and 1D Conv
First try on raw data, then try on the processed version?

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import mat4py # .mat file conversion

from scipy.signal import welch, find_peaks, cwt # Signal Processing

import stumpy # Matrix Profile
from stumpy.floss import _cac # One sided cac

import ot # Optimal Transport

from vmdpy import VMD # Variational Mode Decomposition 

from ruptures.metrics import hausdorff # Quantifying Break Point Discrepancies

# Utils
import time
import os
loc = os.getcwd() 

# Matplotlib Params
import matplotlib
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 16}

matplotlib.rc('font', **font)

sys.path.append("/Users/wang_to/Documents/University/Anomaly_detection/anomalies/carvalho-etal-2019")

import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import SGD, Adam
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LeakyReLU
from keras.models import Sequential
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
from main_feats import featExtract

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [3]:
data_dict = np.load('/Users/wang_to/Documents/University/Anomaly_detection/anomalies/bookshelf/dataset/data_dict.npy',allow_pickle='TRUE').item()
building_features = np.load('/Users/wang_to/Documents/University/Anomaly_detection/anomalies/bookshelf/dataset/building_features.npy',allow_pickle='TRUE').item()


In [5]:
sens1 = data_dict["Sensor1"]

sens = []

num_sens = 25

for i in range(num_sens):
    sens.append(data_dict[f"Sensor{num_sens-1}"])



sens1_data = sens1.iloc[:,:-4]
print(f'{sens1.shape} {sens1_data.shape}')

data_labels = sens1["y_labels"].values
data_labels = np.array([term[0] for term in data_labels])

(270, 8196) (270, 8192)


In [6]:
sens = np.concatenate(sens)

In [9]:
sens[0, -2]

'D00'

In [28]:
np.unique(sens[:, -2])

TypeError: '<' not supported between instances of 'int' and 'str'

In [32]:
sens_data = sens[:,:-4].astype('float32')


In [33]:
sens = np.where(sens == 'D00', 0, sens)
sens = np.where(sens == 'DB0', 1, sens)
sens = np.where(sens == 'DBB', 2, sens)
sens = np.where(sens == 'DHT', 3, sens)
sens = np.where(sens == 'D05', 4, sens)
sens = np.where(sens == 'D10', 5, sens)

sens_labels = sens[:, -4]

In [37]:
sens_damage_levels =sens[:, -2]

In [41]:
np.save('sens_damage_levels.npy', sens_damage_levels)

In [40]:
sens_damage_levels

array([0, 0, 0, ..., 2, 2, 2], dtype=object)

In [22]:
sens_data

array([[ 0.0070295, -0.27185  , -0.38852  , ..., -0.27888  , -0.43123  ,
        -0.16628  ],
       [ 0.0060832, -0.022711 , -0.018926 , ..., -0.60062  , -0.58629  ,
        -0.56263  ],
       [ 0.34215  ,  0.26185  ,  0.19061  , ...,  0.04096  , -0.15681  ,
        -0.29619  ],
       ...,
       [ 0.39527  ,  0.40501  ,  0.14911  , ..., -0.1975   , -0.26699  ,
        -0.27307  ],
       [ 0.32065  ,  0.47071  ,  0.45962  , ...,  0.12423  ,  0.20277  ,
         0.2048   ],
       [-0.013518 ,  0.21778  ,  0.1268   , ..., -0.13275  , -0.074351 ,
        -0.31836  ]], dtype=float32)

In [43]:
sens_data.shape

(6750, 8192)

In [21]:
sens_damage_levels

array([0, 0, 0, ..., 2, 2, 2], dtype=object)

In [24]:
np.save("sens_damage_levels.npy", sens_damage_levels)

In [32]:
np.save("sens_data.npy", sens_data)
np.save("sens_labels.npy", sens_labels)

In [22]:
sens.shape

(6750, 8196)

In [4]:
vmd_sens1 = building_features["Sensor1"]  # 5 vmd, 513 long time series each 


In [5]:
sens1_data.values.shape

(270, 8192)

In [6]:
# data training setup 
scaler = MinMaxScaler()
scaler.fit(sens1_data)
sens1_data_normalised = scaler.transform(sens1_data)

train_X = sens1_data.values.reshape((-1, sens1_data.shape[1], 1))
train_X_norm = sens1_data_normalised.reshape((-1, sens1_data_normalised.shape[1], 1))

/Users/wang_to/opt/anaconda3/envs/end2end/lib/python3.7/site-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/Users/wang_to/opt/anaconda3/envs/end2end/lib/python3.7/site-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [7]:
print(f'{train_X_norm.shape}, {train_X.shape}')


(270, 8192, 1), (270, 8192, 1)


In [8]:
from keras.utils import to_categorical
data_category = to_categorical(data_labels)

array([[0.0070295],
       [-0.27185],
       [-0.38852],
       ...,
       [-0.27888],
       [-0.43123],
       [-0.16628]], dtype=object)

In [9]:
np.save("data_labels.npy", data_labels)
np.save("data_category.npy", data_category)
np.save("train_X.npy", train_X)

In [55]:
sens_data[0]

array([0.0070295, -0.27185, -0.38852, ..., -0.27888, -0.43123, -0.16628],
      dtype=object)

In [57]:
train_X = sens_data
train_X = train_X.reshape((-1, train_X.shape[1], 1))
train_X = train_X.astype('float32')
data_category = sens_labels

In [59]:
train_X.shape

(6750, 8192, 1)

In [63]:
np.save("sens_data.npy", train_X)
np.save("sens_Y.npy", data_category)

# One-hot encoding the damage levels
D00, DB0, DBB

In [1]:
data_dict

NameError: name 'data_dict' is not defined

In [61]:
num_epochs=50

optim = Adam(lr=1e-4)

num_folds = 5
kfold = KFold(n_splits = num_folds, shuffle=True, random_state=1337)
kfold.get_n_splits(train_X)
fold_no = 1 

scores_per_fold = []

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

# define model 
batch_size = 128
verbose = 1
input_shape = (batch_size, train_X.shape[1])

l1_reg = 1e-6
l2_reg = 1e-6
class conv1d_nn(tf.keras.Model):
    def __init__(self, n_outputs): # just get it working first. Set n_outputs = 1 for 1-class categorisation
        super(conv1d_nn, self).__init__()
        
        self.conv_in = Conv1D(
            filters=64,
            input_shape=input_shape,
            kernel_size=32, 
            activation='relu', 
            name='in'
        )
        self.conv1 = Conv1D(
            filters=32, 
            kernel_size=16, 
            activation='relu',
            bias_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            activity_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)    
        )
        self.conv2 = Conv1D(
            filters=32,
            kernel_size=8,
            activation='relu',
            bias_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            activity_regularizer=l1_l2(l1=l1_reg, l2=l2_reg),
            kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)    
        )
        self.drop = Dropout(0.5)
        self.maxPool = MaxPooling1D(pool_size=2, strides=2)
        self.flat = Flatten()
        self.D1 = Dense(100, activation = 'relu')
        self.D2 = Dense(n_outputs, activation='sigmoid')

    def call(self, inputs, training=False):
        x = self.conv_in(inputs)

        x = self.conv1(x)
        x = self.maxPool(x)

        x = self.conv2(x)
        x = self.maxPool(x)

        x = self.flat(x)

        #x = self.D1(x)
        # if training==True:
        #     x = self.drop(x)
        x = self.D2(x)
        return(x)
        

for train_index, test_index in kfold.split(train_X):
    print(f'Fold no: {fold_no}')
    X_train, X_test = train_X[train_index], train_X[test_index]
    Y_train, Y_test = data_category[train_index], data_category[test_index]

    model = conv1d_nn(n_outputs=1)

    model.compile(optimizer='adam', loss='categorical_crossentropy', 
    metrics=METRICS)
    print('-------------------------------------------')
    print(f'Fitting model on X_train, Y_train:')
    history = model.fit(X_train, Y_train, 
        batch_size=batch_size, 
        epochs=num_epochs,
        verbose=verbose)

    print(model.summary())

    scores = model.evaluate(X_test, Y_test, verbose=1)
    print(scores)
    print(model.metrics_names)
    zipped = zip(model.metrics_names, scores)
    scores_per_fold.append(zipped)
    
    print(f"Score for fold {fold_no}: {[zips for zips in zipped]}.")

    fold_no += 1



Fold no: 1
-------------------------------------------
Fitting model on X_train, Y_train:
Epoch 1/50
1792/5400 [========>.....................] - ETA: 2:58 - loss: 5.4216e-08 - tp: 447.0000 - fp: 544.0000 - tn: 433.0000 - fn: 368.0000 - accuracy: 0.4911 - precision: 0.4511 - recall: 0.5485 - auc: 0.4974 - prc: 0.4242

KeyboardInterrupt: 